# Задание 3

**Выгрузите организации, где число сотрудников в 2020 году 15 и больше:**
* если за 2020 год данных по сотрудникам нет, то возьмите за 2019 год;
* отсортируйте по возрастанию числа сотрудников;  
* если число сотрудников одинаковое, то организации должны идти по убыванию ID.

## Описание датасетов

#### main.csv - *основная информация о предприятиях*
- ID - уникальный номер объекта;
- Name - название;
- Address - адрес

#### net_property.csv - *указание на сетевые предприятия*
- ID - уникальный номер объекта;
- IsNetObject - метка сетевого предприятия;

#### seats_property.csv - *количество посадочных мест*
- ID - уникальный номер объекта;
- SeatsCount - количество посадочных мест;
- ver_ID - номер версии записи

#### type_property.csv - *информация о специализации: столовая, закусочная, кафе, иное*
- ID - уникальный номер объекта;
- TypeObject - тип объекта;
- is_actual - актуальность записи

#### indicators_db.sqlite3 - *SQL-база с данными о выручке предприятий и числе сотрудников*
- данные за 2019 г.;
- данные за 2020-2022 гг.

In [2]:
import pandas as pd
import numpy as np

### Чтение датасетов

In [3]:
df1 = pd.read_csv('main.csv')
df2 = pd.read_csv('net_property.csv')
df3 = pd.read_csv('seats_property.csv')
df4 = pd.read_csv('type_property.csv')

In [4]:
import sqlalchemy as sql

In [5]:
con = sql.create_engine('sqlite:///indicators_db.sqlite3')

In [6]:
df = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';", con)

In [7]:
df

,type,name,tbl_name,rootpage,sql
0,table,org_indicators_2019,org_indicators_2019,2,CREATE TABLE org_indicators_2019 (\r\n\tID BIG...
1,table,org_indicators_2020_2022,org_indicators_2020_2022,490,CREATE TABLE org_indicators_2020_2022 (\r\n\tI...


In [8]:
df_sql_select_19 = pd.read_sql('select * from org_indicators_2019', con)

In [9]:
df_sql_select_20_22 = pd.read_sql('select * from org_indicators_2020_2022', con)

### Выборка необходимых колонок и разворот через pivot

In [10]:
df_19 = df_sql_select_19[df_sql_select_19['INDICATOR_NAME']  == 'Число сотрудников']

In [11]:
df_19_piv = df_19.pivot(index = 'ID', columns = ['YEAR','INDICATOR_NAME'], values = 'VALUE')

In [12]:
df_20_22 = df_sql_select_20_22[df_sql_select_20_22['INDICATOR_NAME']  == 'Число сотрудников']

In [13]:
df_20_22_piv = df_20_22.pivot(index = 'ID', columns = ['YEAR','INDICATOR_NAME'], values = 'VALUE')

In [14]:
df_20_22_piv

YEAR,2020,2021,2022
INDICATOR_NAME,Число сотрудников,Число сотрудников,Число сотрудников
ID,,,
19630,NaN,15.0,9.0
19643,17.0,NaN,NaN
19647,16.0,15.0,NaN
19657,NaN,8.0,12.0
19677,17.0,9.0,NaN
...,...,...,...
375884,8.0,10.0,7.0
375933,NaN,16.0,NaN


### Удаление уровня "YEAR" и переименование колонок

In [15]:
testing_1 = df_19_piv.droplevel([0], axis = 1)
testing_1.columns.names = [None]
testing_1.to_csv('test_19.csv')
testing_1 = pd.read_csv('test_19.csv')
testing_1 = testing_1.rename(columns = {'Число сотрудников':2019})

In [16]:
testing_2 = df_20_22_piv.droplevel([0], axis = 1)
testing_2.columns.names = [None]
testing_2.to_csv('test_20.csv')
testing_2 = pd.read_csv('test_20.csv')
rename = {'Число сотрудников':2020, 'Число сотрудников.1':2021, 'Число сотрудников.2':2022}
testing_2 = testing_2.rename(columns = rename)

In [17]:
testing_2 = testing_2.drop([2021,2022], axis = 1)

In [18]:
testing = testing_1.merge(testing_2, how = 'outer')

In [19]:
testing_drop = testing.dropna(thresh = 2)

In [20]:
testing_drop

,ID,2019,2020
0,19630,7.0,NaN
1,19647,14.0,16.0
2,19681,18.0,16.0
3,19688,18.0,16.0
4,19708,6.0,14.0
...,...,...,...
9851,375760,NaN,10.0
9852,375786,NaN,7.0
9853,375822,NaN,13.0
9855,375848,NaN,12.0


In [21]:
test_melt = testing_drop.melt('ID').dropna()

In [22]:
test_sort = test_melt.sort_values(by = 'variable', ascending = True)

In [23]:
test_last = test_melt.drop_duplicates(['ID'], keep = 'last')

In [24]:
test_final = test_last[test_last['value'] >= 15].drop('variable', axis = 1)

In [25]:
renaming = {'ID':'Идентификатор', 'value':'Сотрудники 2019_2020'}

In [26]:
result = test_final.rename(columns = renaming).sort_values(by = ['Сотрудники 2019_2020','Идентификатор'], 
                                                           ascending = [True, False])

In [27]:
result['Сотрудники 2019_2020'] = result['Сотрудники 2019_2020'].astype(int)

In [28]:
result.to_csv('count_employees.csv', index = False)

In [29]:
result

,Идентификатор,Сотрудники 2019_2020
17770,375648,15
17766,375575,15
15511,375534,15
6615,375320,15
17757,375119,15
...,...,...
8972,20838,19
66,20503,19
15534,20315,19
8928,20045,19
